## Create Responder Table of BLAST results

In [1]:
from IPython.display import Image, FileLink
import pandas as pd
import pandas.rpy.common as com

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(data.table)
library(dplyr)

data.table 1.9.4  For help type: ?data.table
*** NB: by=.EACHI is now explicit. See README to restore previous behaviour.

Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, last

The following object is masked from ‘package:stats’:

    filter

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



###This file is from the [sparsity notebook](./Sparsity_make_l2fc_table.ipynb).

In [4]:
%%R
df.l2fc = read.csv("data/l2fc_table.csv")

In [5]:
%%R

df.l2fc.SS = tbl_df(df.l2fc) %.% 
        filter(Day == "30") %.%
        filter(Treatment == "13CCSS")
df.l2fc.SS

Source: local data frame [632 x 15]

   sparsity_threshold Day Treatment      OTU log2FoldChange            p
1                0.55  30    13CCSS  OTU.783     -0.3370825 9.965741e-01
2                0.55  30    13CCSS   OTU.79      0.7598182 4.889860e-01
3                0.55  30    13CCSS  OTU.304      0.4183648 8.089837e-01
4                0.55  30    13CCSS   OTU.50      0.7289694 5.252368e-01
5                0.55  30    13CCSS  OTU.382      2.9833574 1.370092e-07
6                0.55  30    13CCSS  OTU.465      0.5515994 6.887540e-01
7                0.55  30    13CCSS  OTU.162      0.2116710 9.204188e-01
8                0.55  30    13CCSS OTU.1084      1.1894339 1.366258e-01
9                0.55  30    13CCSS  OTU.525      1.9020665 2.101013e-03
10               0.55  30    13CCSS  OTU.985      1.1943492 1.358942e-01
..                ... ...       ...      ...            ...          ...
Variables not shown: Rank1 (fctr), Rank2 (fctr), Rank3 (fctr), Rank4 (fctr),
  Rank5 (f

In [6]:
%%R
df.blast = tbl_df(read.table("data/blast_out_otusn_LTP115.m6",
                  sep = "\t"))
names(df.blast) = c("OTU", "acc", "pid", "alnlen", "mismatches", "gaps", 
                    "qstart", "qend", "sstart", "send", "evalue", "bit")

df.blast

Source: local data frame [1,919,502 x 12]

     OTU      acc pid alnlen mismatches gaps qstart qend sstart send evalue bit
1  OTU.1 AF235091 100    219          0    0      1  219    508  726 2e-113 405
2  OTU.1 AF330692 100    219          0    0      1  219    502  720 2e-113 405
3  OTU.1 AB588633 100    219          0    0      1  219    539  757 2e-113 405
4  OTU.1   X80741 100    219          0    0      1  219    532  750 2e-113 405
5  OTU.1 AM176541 100    219          0    0      1  219    541  759 2e-113 405
6  OTU.1   X80740 100    219          0    0      1  219    533  751 2e-113 405
7  OTU.1   X83408 100    219          0    0      1  219    529  747 2e-113 405
8  OTU.1 AB279889 100    219          0    0      1  219    538  756 2e-113 405
9  OTU.1 GQ406811 100    219          0    0      1  219    492  710 2e-113 405
10 OTU.1 AB279890 100    219          0    0      1  219    542  760 2e-113 405
..   ...      ... ...    ...        ...  ...    ...  ...    ...  ...    ... .

In [7]:
%%R
df.tax = tbl_df(read.csv("/var/seq_data/silva/silva_blastdb/full_names.csv"))
df.tax

Source: local data frame [1,426,450 x 2]

        acc                           full_name
1  AX003092          Enterococcus casseliflavus
2  AX044029              Neisseria meningitidis
3  EU271959            Myzocytiopsis intermedia
4  AX039535            Dehalococcoides mccartyi
5  EU273602                   Acorus americanus
6  AX175616                Marinomonas communis
7  AX044033              Neisseria meningitidis
8  AB000389         Pseudoalteromonas elyakovii
9  EU271960          Myzocytiopsis sp. venatrix
10 AB001439 Pseudomonas syringae pv. actinidiae
..      ...                                 ...


In [8]:
%%R
df.blast = left_join(df.blast, df.tax)

Joining by: "acc"


In [9]:
%%R
df.blast = left_join(df.blast, df.l2fc.SS %>% select(OTU, padj, log2FoldChange, Treatment, Rank2, Rank3, Rank4))

Joining by: "OTU"


In [10]:
%%R
df.top.hits = df.blast %>%
    group_by(OTU) %>%
    mutate(bit.rank = rank(desc(bit), ties.method = "min")) %>%
    filter(bit.rank == 1) %>%
    group_by()

In [11]:
%%R
df.top.hits

Source: local data frame [10,521 x 20]

     OTU      acc pid alnlen mismatches gaps qstart qend sstart send evalue bit
1  OTU.1 AF235091 100    219          0    0      1  219    508  726 2e-113 405
2  OTU.1 AF330692 100    219          0    0      1  219    502  720 2e-113 405
3  OTU.1 AB588633 100    219          0    0      1  219    539  757 2e-113 405
4  OTU.1   X80741 100    219          0    0      1  219    532  750 2e-113 405
5  OTU.1 AM176541 100    219          0    0      1  219    541  759 2e-113 405
6  OTU.1   X80740 100    219          0    0      1  219    533  751 2e-113 405
7  OTU.1   X83408 100    219          0    0      1  219    529  747 2e-113 405
8  OTU.1 AB279889 100    219          0    0      1  219    538  756 2e-113 405
9  OTU.1 GQ406811 100    219          0    0      1  219    492  710 2e-113 405
10 OTU.1 AB279890 100    219          0    0      1  219    542  760 2e-113 405
..   ...      ... ...    ...        ...  ...    ...  ...    ...  ...    ... ...


In [12]:
%%R
FDR = 0.10

df.table = df.top.hits %>%
    filter(padj <= FDR) %>%
    group_by(OTU) %>%
    summarize(hits = paste(full_name, collapse = "|"), 
              pid = first(pid), 
              log2FoldChange = max(log2FoldChange), 
              Phylum = first(as.character(Rank2)),
              Class = first(as.character(Rank3)),
              Order = first(as.character(Rank4)),
              Treatment = first(as.character(Treatment))) %>%
    arrange(Phylum, Class, Order, desc(log2FoldChange))

In [13]:
%%R
df.table

Source: local data frame [29 x 8]

        OTU
1   OTU.569
2   OTU.382
3   OTU.525
4    OTU.64
5    OTU.98
6  OTU.4322
7   OTU.285
8   OTU.766
9   OTU.206
10   OTU.73
..      ...
Variables not shown: hits (chr), pid (dbl), log2FoldChange (dbl), Phylum (chr),
  Class (chr), Order (chr), Treatment (chr)


In [14]:
df_table = com.load_data("df.table")

In [15]:
df_table.head()

,OTU,hits,pid,log2FoldChange,Phylum,Class,Order,Treatment
1,OTU.569,Moorella sp. 64_FGQ|Ammoniphilus oxalaticus,84.16,2.151863,Acidobacteria,Candidatus_Solibacter,uncultured_bacterium,13CCSS
2,OTU.382,Ohtaekwangia koreensis,89.19,2.983357,Bacteroidetes,Cytophagia,Cytophagales,13CCSS
3,OTU.525,Cytophaga hutchinsonii ATCC 33406|Cytophaga hu...,98.63,1.902067,Bacteroidetes,Cytophagia,Cytophagales,13CCSS
4,OTU.64,Herpetosiphon geysericola,89.50,2.780829,Chloroflexi,Herpetosiphonales,Herpetosiphonaceae,13CCSS
5,OTU.98,Herpetosiphon geysericola,88.18,2.558323,Chloroflexi,Herpetosiphonales,Herpetosiphonaceae,13CCSS


In [16]:
def list_genera(l):
    if len(l.split("|")) > 10:
        names = l.split("|")
        names_abbr = []
        seen = []
        for name in names:
            genus, species = name.split(" ", 1)
            if genus in seen:
                continue
            else:
                seen.append(genus)
                names_abbr.append("\mbox{" + "\\textit{" + genus + " spp." + "}" + "}")
        return ", ".join(names_abbr) 
    elif len(l.split("|")) > 1:
        return  ", ".join(["\mbox{" + "\\textit{" + i + "}" + "}" for i in set(l.split("|"))])
    else:
        return "\mbox{" + "\\textit{" + l + "}" + "}"
    
def get_latex(row):
    latex = ""
    OTU = row["OTU"]
    pid = row["pid"]
    
    if pid >= 95:
        genera = list_genera(row["hits"])
    else:
        genera = "{No hits of at least 95\% identity}"

    tax = " ".join(["\mbox{" + "\\textit{" + i + "}" + "}" for i in row[["Phylum","Class","Order"]]]).replace("_","-")
    
    l2fc = pd.Series(row["log2FoldChange"]).round(2).values[0]
    #acc = df.acc.iloc[0]
    return OTU + " & " + str(l2fc) + " & " + genera + " & " + str(pid) + " & " + tax + r" \\ " + r"\midrule" + "\n"

In [34]:
s = df_table[df_table["Treatment"]=="13CCSS"].apply(get_latex, axis=1)

table_framework = r"""
\documentclass[10pt]{article}
\usepackage{multirow, array, booktabs, longtable, threeparttablex}
\usepackage{array}
\usepackage{enumitem}
\usepackage{chngcntr}
\counterwithin{table}{section}
\usepackage[margin=1cm]{caption}
\newcolumntype{P}[1]{>{\raggedright\arraybackslash}p{#1}}

\usepackage{geometry} 
\geometry{tmargin=1cm, bmargin=1cm, lmargin=0.25cm, rmargin=0.25cm} 

\begin{document}
\setcounter{table}{1}
\setcounter{section}{3}

\pagestyle{empty}

\begin{ThreePartTable}
\begin{TableNotes}
\item[a] Maximum observed $log_{2}$ of fold change. 
\item[b] Against Living Tree Project database.
\item[c] Annotation from Silva database assigned during OTU binning (see methods).
\end{TableNotes}

\begin{longtable}{lrP{5cm}rP{7cm}}

\caption{$^{13}$C-cellulose responders in the simple treatment} \\
\toprule 
    \textbf{OTU ID} & 
    \textbf{Fold change} \tnote{a} &  
    \textbf{Top BLAST hits} \tnote{b}& 
    \textbf{BLAST \%%ID} \tnote{b}& 
    \textbf{Phylum;Class;Order} \tnote{c}\\
\midrule
\endfirsthead

\multicolumn{3}{c}
{{\tablename\ \thetable{} -- continued from previous page}} \\
\midrule
    \textbf{OTU ID} & 
    \textbf{Fold change} & 
    \textbf{Top BLAST hits} & 
    \textbf{BLAST \%%ID} & 
    \textbf{Phylum;Class;Order} \\
\midrule
\endhead
    %s
\bottomrule
\insertTableNotes
\end{longtable}

\end{ThreePartTable}
 
\end{document}"""%"".join(s.values)

with open("data/table_SS.tex", "w") as out:
    out.write(table_framework)

In [35]:
!latex data/table_SS.tex >/dev/null
!dvipdf table_SS.dvi figs/LTP_blast_table_SS.pdf

In [36]:
!convert -density 300 -trim  +repage figs/LTP_blast_table_SS.pdf -quality 100 figs/LTP_blast_table_SS.png

In [37]:
FileLink("figs/LTP_blast_table_SS.pdf")

/home/ashley/single_substrate/figs/LTP_blast_table_SS.pdf

In [38]:
FileLink("figs/LTP_blast_table_SS.png")

/home/ashley/single_substrate/figs/LTP_blast_table_SS.png

In [39]:
FileLink("data/table_SS.tex")

/home/ashley/single_substrate/data/table_SS.tex